# 정수 인코딩 (Integer Encoding)

- 자연어 처리는 텍스트 데이터를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 만드는 것이 핵심
- 정수 인코딩을 수행하여 텍스트 데이터에 고유한 인덱스를 부여 (1~5,000)
- 이러한 인코딩 과정은 전처리 과정에서 필수적이며 각 단어의 등장 빈도에 따라 인덱스를 부여하는 것이 일반적
- 단어 수를 5,000으로 제한하는 것은 모델 학습에 필요한 메모리와 계산 자원을 줄이기 위함 (등장 빈도가 낮은 단어는 제외하고 상위 5,000개 단어만 선택하는 것이 일반적)

In [1]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

### 인코딩 처리

##### 토큰화 + 정제/정규화

In [3]:
import nltk

nltk.download('punkt')        # 토큰화 리소스
nltk.download('punkt_tab')    # punkt 관련 추가 리소스
nltk.download('stopwords')    # 불용어 목록

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# NLTK 문장/단어 토큰화 + 불용어 제거 + 단어 빈도 사전 구축
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# 문장 토큰화
sentences = sent_tokenize(raw_text)

# 영어 불용어 리스트
en_stopwords = stopwords.words('english')

# 단어사전 (key=단어, value=빈도)
vocab = {}    # 단어별 등장 빈도를 저장할 딕셔너리

# 토큰화/정제/정규화 처리 결과
preprocessed_sentences = []    # 전처리된 문장(토큰 리스트)들을 모을 리스트

for sentence in sentences:
    sentence = sentence.lower()         # 문장을 소문자로 변환
    tokens = word_tokenize(sentence)    # 문장을 단어(토큰) 리스트로 분리 
    tokens = [token for token in tokens if token not in en_stopwords]    # 불용어 제거
    tokens = [token for token in tokens if len(token) > 2]    # 길이 3 이상인 토큰만 남김 (노이즈 축소)

    for token in tokens:
        if token not in vocab:    # 처음 등장한 단어이면
            vocab[token] = 1      # 해당 단어를 키로, 빈도수 1로 저장
        else:
            vocab[token] += 1     # 해당 단어에 빈도수 +1
    
    preprocessed_sentences.append(tokens)    # 문장별 전처리 토큰 결과를 리스트에 저장

print(preprocessed_sentences)
print(len(vocab))                # 현재 vocab에는 고유한 단어(unique tokens)만 들어있다.

[['little', 'prince', 'written', 'antoine', 'saint-exupéry', 'poetic', 'tale', 'young', 'prince', 'travels', 'home', 'planet', 'earth'], ['story', 'begins', 'pilot', 'stranded', 'sahara', 'desert', 'plane', 'crashes'], ['trying', 'fix', 'plane', 'meets', 'mysterious', 'young', 'boy', 'little', 'prince'], ['little', 'prince', 'comes', 'small', 'asteroid', 'called', 'b-612', 'lives', 'alone', 'rose', 'loves', 'deeply'], ['recounts', 'journey', 'pilot', 'describing', 'visits', 'several', 'planets'], ['planet', 'inhabited', 'different', 'character', 'king', 'vain', 'man', 'drunkard', 'businessman', 'geographer', 'fox'], ['encounters', 'prince', 'learns', 'valuable', 'lessons', 'love', 'responsibility', 'nature', 'adult', 'behavior'], ['earth', 'little', 'prince', 'meets', 'various', 'creatures', 'including', 'fox', 'teaches', 'relationships', 'importance', 'taming', 'means', 'building', 'ties', 'others'], ['fox', 'famous', 'line', 'become', 'responsible', 'forever', 'tamed', 'resonates', '

len(vocab)은 전체 코퍼스의 어휘 크기(vocabulary size) 이다.

##### 빈도수 기반 정제

In [7]:
vocab

{'little': 6,
 'prince': 9,
 'written': 1,
 'antoine': 1,
 'saint-exupéry': 1,
 'poetic': 1,
 'tale': 1,
 'young': 2,
 'travels': 1,
 'home': 1,
 'planet': 2,
 'earth': 2,
 'story': 2,
 'begins': 1,
 'pilot': 4,
 'stranded': 1,
 'sahara': 1,
 'desert': 1,
 'plane': 2,
 'crashes': 1,
 'trying': 1,
 'fix': 1,
 'meets': 2,
 'mysterious': 1,
 'boy': 1,
 'comes': 1,
 'small': 1,
 'asteroid': 2,
 'called': 1,
 'b-612': 1,
 'lives': 1,
 'alone': 1,
 'rose': 3,
 'loves': 1,
 'deeply': 1,
 'recounts': 1,
 'journey': 1,
 'describing': 1,
 'visits': 1,
 'several': 1,
 'planets': 1,
 'inhabited': 1,
 'different': 1,
 'character': 1,
 'king': 1,
 'vain': 1,
 'man': 1,
 'drunkard': 1,
 'businessman': 1,
 'geographer': 1,
 'fox': 3,
 'encounters': 1,
 'learns': 1,
 'valuable': 1,
 'lessons': 2,
 'love': 2,
 'responsibility': 1,
 'nature': 1,
 'adult': 1,
 'behavior': 1,
 'various': 1,
 'creatures': 1,
 'including': 1,
 'teaches': 1,
 'relationships': 1,
 'importance': 2,
 'taming': 1,
 'means': 1,
 '

In [ ]:
# vocab (단어, 빈도) 튜플을 빈도(item[1] 기준으로 내림차순 정렬)
vocab_sorted = sorted(vocab.items(), key=lambda item: item[1], reverse=True)
vocab_sorted

[('prince', 9),
 ('little', 6),
 ('pilot', 4),
 ('rose', 3),
 ('fox', 3),
 ('young', 2),
 ('planet', 2),
 ('earth', 2),
 ('story', 2),
 ('plane', 2),
 ('meets', 2),
 ('asteroid', 2),
 ('lessons', 2),
 ('love', 2),
 ('importance', 2),
 ('written', 1),
 ('antoine', 1),
 ('saint-exupéry', 1),
 ('poetic', 1),
 ('tale', 1),
 ('travels', 1),
 ('home', 1),
 ('begins', 1),
 ('stranded', 1),
 ('sahara', 1),
 ('desert', 1),
 ('crashes', 1),
 ('trying', 1),
 ('fix', 1),
 ('mysterious', 1),
 ('boy', 1),
 ('comes', 1),
 ('small', 1),
 ('called', 1),
 ('b-612', 1),
 ('lives', 1),
 ('alone', 1),
 ('loves', 1),
 ('deeply', 1),
 ('recounts', 1),
 ('journey', 1),
 ('describing', 1),
 ('visits', 1),
 ('several', 1),
 ('planets', 1),
 ('inhabited', 1),
 ('different', 1),
 ('character', 1),
 ('king', 1),
 ('vain', 1),
 ('man', 1),
 ('drunkard', 1),
 ('businessman', 1),
 ('geographer', 1),
 ('encounters', 1),
 ('learns', 1),
 ('valuable', 1),
 ('responsibility', 1),
 ('nature', 1),
 ('adult', 1),
 ('behavio

In [ ]:
# 단어를 정수 인덱스로 매핑하는 사전 생성 (1부터 시작. key=단어, value=인덱스)
word_to_idx = {word: i+1 for i, (word, cnt) in enumerate(vocab_sorted)}
word_to_idx

{'prince': 1,
 'little': 2,
 'pilot': 3,
 'rose': 4,
 'fox': 5,
 'young': 6,
 'planet': 7,
 'earth': 8,
 'story': 9,
 'plane': 10,
 'meets': 11,
 'asteroid': 12,
 'lessons': 13,
 'love': 14,
 'importance': 15,
 'written': 16,
 'antoine': 17,
 'saint-exupéry': 18,
 'poetic': 19,
 'tale': 20,
 'travels': 21,
 'home': 22,
 'begins': 23,
 'stranded': 24,
 'sahara': 25,
 'desert': 26,
 'crashes': 27,
 'trying': 28,
 'fix': 29,
 'mysterious': 30,
 'boy': 31,
 'comes': 32,
 'small': 33,
 'called': 34,
 'b-612': 35,
 'lives': 36,
 'alone': 37,
 'loves': 38,
 'deeply': 39,
 'recounts': 40,
 'journey': 41,
 'describing': 42,
 'visits': 43,
 'several': 44,
 'planets': 45,
 'inhabited': 46,
 'different': 47,
 'character': 48,
 'king': 49,
 'vain': 50,
 'man': 51,
 'drunkard': 52,
 'businessman': 53,
 'geographer': 54,
 'encounters': 55,
 'learns': 56,
 'valuable': 57,
 'responsibility': 58,
 'nature': 59,
 'adult': 60,
 'behavior': 61,
 'various': 62,
 'creatures': 63,
 'including': 64,
 'teaches'

인덱스 i, 단어 word만 사용. 빈도 cnt는 사용하지 않음  
사용안하면 보통 for i, (word, _) in enumerate(vocab_sorted)로 사용하는게 default

인덱스를 1부터 시작하는 이유는 0을 padding이나 OOV 같은 특수 토큰 용도로 비워둔 것이다.

In [9]:
# 인덱스를 이용해 단어로 되돌리는 사전 생성 (1부터 시작. key=인덱스, value=단어)
idx_to_word = {i+1 : word for i, (word, _) in enumerate(vocab_sorted)}
idx_to_word

{1: 'prince',
 2: 'little',
 3: 'pilot',
 4: 'rose',
 5: 'fox',
 6: 'young',
 7: 'planet',
 8: 'earth',
 9: 'story',
 10: 'plane',
 11: 'meets',
 12: 'asteroid',
 13: 'lessons',
 14: 'love',
 15: 'importance',
 16: 'written',
 17: 'antoine',
 18: 'saint-exupéry',
 19: 'poetic',
 20: 'tale',
 21: 'travels',
 22: 'home',
 23: 'begins',
 24: 'stranded',
 25: 'sahara',
 26: 'desert',
 27: 'crashes',
 28: 'trying',
 29: 'fix',
 30: 'mysterious',
 31: 'boy',
 32: 'comes',
 33: 'small',
 34: 'called',
 35: 'b-612',
 36: 'lives',
 37: 'alone',
 38: 'loves',
 39: 'deeply',
 40: 'recounts',
 41: 'journey',
 42: 'describing',
 43: 'visits',
 44: 'several',
 45: 'planets',
 46: 'inhabited',
 47: 'different',
 48: 'character',
 49: 'king',
 50: 'vain',
 51: 'man',
 52: 'drunkard',
 53: 'businessman',
 54: 'geographer',
 55: 'encounters',
 56: 'learns',
 57: 'valuable',
 58: 'responsibility',
 59: 'nature',
 60: 'adult',
 61: 'behavior',
 62: 'various',
 63: 'creatures',
 64: 'including',
 65: 'teac

idx_to_word는 정수 인덱스를 다시 단어로 복원할 때(디코딩/해석) 사용

In [ ]:
word_to_idx.items()

dict_items([('prince', 1), ('little', 2), ('pilot', 3), ('rose', 4), ('fox', 5), ('young', 6), ('planet', 7), ('earth', 8), ('story', 9), ('plane', 10), ('meets', 11), ('asteroid', 12), ('lessons', 13), ('love', 14), ('importance', 15), ('written', 16), ('antoine', 17), ('saint-exupéry', 18), ('poetic', 19), ('tale', 20), ('travels', 21), ('home', 22), ('begins', 23), ('stranded', 24), ('sahara', 25), ('desert', 26), ('crashes', 27), ('trying', 28), ('fix', 29), ('mysterious', 30), ('boy', 31), ('comes', 32), ('small', 33), ('called', 34), ('b-612', 35), ('lives', 36), ('alone', 37), ('loves', 38), ('deeply', 39), ('recounts', 40), ('journey', 41), ('describing', 42), ('visits', 43), ('several', 44), ('planets', 45), ('inhabited', 46), ('different', 47), ('character', 48), ('king', 49), ('vain', 50), ('man', 51), ('drunkard', 52), ('businessman', 53), ('geographer', 54), ('encounters', 55), ('learns', 56), ('valuable', 57), ('responsibility', 58), ('nature', 59), ('adult', 60), ('behav

In [11]:
# 상위 vocab_size 단어만 남기는 단어사전 축소
vocab_size = 15

word_to_idx = {word : index for word, index in word_to_idx.items() if index <= vocab_size}    # 인덱스가 vocab_size 이하인 단어만 필터링해 축소 
word_to_idx         # 상위 15개 단어 확인

{'prince': 1,
 'little': 2,
 'pilot': 3,
 'rose': 4,
 'fox': 5,
 'young': 6,
 'planet': 7,
 'earth': 8,
 'story': 9,
 'plane': 10,
 'meets': 11,
 'asteroid': 12,
 'lessons': 13,
 'love': 14,
 'importance': 15}

In [ ]:
# OOV(사전에 없는 단어) 토큰 인덱스 추가
word_to_idx['OOV'] = len(word_to_idx) + 1  # 현재 단어사전 크기 다음 번호를 OOV 인덱스로 추가
word_to_idx

{'prince': 1,
 'little': 2,
 'pilot': 3,
 'rose': 4,
 'fox': 5,
 'young': 6,
 'planet': 7,
 'earth': 8,
 'story': 9,
 'plane': 10,
 'meets': 11,
 'asteroid': 12,
 'lessons': 13,
 'love': 14,
 'importance': 15,
 'OOV': 16}

이후 사전에 없는 단어는 word_to_idx['OOV'] 인덱스로 치환해서 처리한다.

##### 정수 인코딩 (== 수열처리)

In [ ]:
# 전처리 문장을 정수 인덱스 시퀀스로 인코딩 (OOV 처리 포함)
encoded_sentences = []          # 문장별 인코딩 결과(정수 리스트)를 저장할 리스트
oov_idx = word_to_idx['OOV']    # 사전에 없는 단어(OOV)에 대응할 인덱스 값

for sentence in preprocessed_sentences:    # 전처리된 문장(토큰 리스트)들을 순회
    encoded_sentence = [word_to_idx.get(token, oov_idx) for token in sentence]    # 토큰을 인덱스로 변환 (없으면 oov_idx로 대체)
    print(sentence)            # 원래의 토큰 문장 출력
    print(encoded_sentence)    # 인코딩된 인덱스 시퀀스 출력
    print()
    encoded_sentences.append(encoded_sentence)    # 인코딩 결과는 전체 리스트에 누적 저장

['little', 'prince', 'written', 'antoine', 'saint-exupéry', 'poetic', 'tale', 'young', 'prince', 'travels', 'home', 'planet', 'earth']
[2, 1, 16, 16, 16, 16, 16, 6, 1, 16, 16, 7, 8]

['story', 'begins', 'pilot', 'stranded', 'sahara', 'desert', 'plane', 'crashes']
[9, 16, 3, 16, 16, 16, 10, 16]

['trying', 'fix', 'plane', 'meets', 'mysterious', 'young', 'boy', 'little', 'prince']
[16, 16, 10, 11, 16, 6, 16, 2, 1]

['little', 'prince', 'comes', 'small', 'asteroid', 'called', 'b-612', 'lives', 'alone', 'rose', 'loves', 'deeply']
[2, 1, 16, 16, 12, 16, 16, 16, 16, 4, 16, 16]

['recounts', 'journey', 'pilot', 'describing', 'visits', 'several', 'planets']
[16, 16, 3, 16, 16, 16, 16]

['planet', 'inhabited', 'different', 'character', 'king', 'vain', 'man', 'drunkard', 'businessman', 'geographer', 'fox']
[7, 16, 16, 16, 16, 16, 16, 16, 16, 16, 5]

['encounters', 'prince', 'learns', 'valuable', 'lessons', 'love', 'responsibility', 'nature', 'adult', 'behavior']
[16, 1, 16, 16, 13, 14, 16, 16, 1

In [14]:
encoded_sentences

[[2, 1, 16, 16, 16, 16, 16, 6, 1, 16, 16, 7, 8],
 [9, 16, 3, 16, 16, 16, 10, 16],
 [16, 16, 10, 11, 16, 6, 16, 2, 1],
 [2, 1, 16, 16, 12, 16, 16, 16, 16, 4, 16, 16],
 [16, 16, 3, 16, 16, 16, 16],
 [7, 16, 16, 16, 16, 16, 16, 16, 16, 16, 5],
 [16, 1, 16, 16, 13, 14, 16, 16, 16, 16],
 [8, 2, 1, 11, 16, 16, 16, 5, 16, 16, 15, 16, 16, 16, 16, 16],
 [5, 16, 16, 16, 16, 16, 16, 16, 1, 16, 4],
 [16, 2, 1, 16, 16, 16, 16, 16, 16, 16],
 [16, 16, 3, 16, 16, 12, 16, 4],
 [9, 16, 3, 16, 13, 16, 2, 1, 16, 16, 16],
 [16, 16, 16, 16, 16, 16, 14, 16, 15, 16, 16, 16, 16]]

### Keras Tokenizer

In [16]:
# keras tokenizer로 단어사전(word_index) 생성 (OOV 포함)
from tensorflow.keras.preprocessing.text import Tokenizer    # keras의 텍스트 토큰화/인덱싱 도구

tokenizer = Tokenizer(num_words=15, oov_token='<OOV>')    # 상위 15개 단어만 사용(필터링), 그 외는 <OOV>로 매핑
tokenizer.fit_on_texts(preprocessed_sentences)    # 문장(토큰 리스트)들을 학습해 단어 빈도 기반 인덱스 사전 생성

tokenizer.word_index    # 코퍼스의 전체 단어에 대해 단어 -> 인덱스 딕셔너리

{'<OOV>': 1,
 'prince': 2,
 'little': 3,
 'pilot': 4,
 'rose': 5,
 'fox': 6,
 'young': 7,
 'planet': 8,
 'earth': 9,
 'story': 10,
 'plane': 11,
 'meets': 12,
 'asteroid': 13,
 'lessons': 14,
 'love': 15,
 'importance': 16,
 'written': 17,
 'antoine': 18,
 'saint-exupéry': 19,
 'poetic': 20,
 'tale': 21,
 'travels': 22,
 'home': 23,
 'begins': 24,
 'stranded': 25,
 'sahara': 26,
 'desert': 27,
 'crashes': 28,
 'trying': 29,
 'fix': 30,
 'mysterious': 31,
 'boy': 32,
 'comes': 33,
 'small': 34,
 'called': 35,
 'b-612': 36,
 'lives': 37,
 'alone': 38,
 'loves': 39,
 'deeply': 40,
 'recounts': 41,
 'journey': 42,
 'describing': 43,
 'visits': 44,
 'several': 45,
 'planets': 46,
 'inhabited': 47,
 'different': 48,
 'character': 49,
 'king': 50,
 'vain': 51,
 'man': 52,
 'drunkard': 53,
 'businessman': 54,
 'geographer': 55,
 'encounters': 56,
 'learns': 57,
 'valuable': 58,
 'responsibility': 59,
 'nature': 60,
 'adult': 61,
 'behavior': 62,
 'various': 63,
 'creatures': 64,
 'including': 

num_words = 15로 설정한 제한은 texts_to_sequences() 등에서 상위 단어만 사용할때 적용한다.

In [ ]:
# 인덱스 -> 단어 사전
tokenizer.index_word    # corpus의 전체 단어에 대한 인덱스 -> 단어 딕셔너리

{1: '<OOV>',
 2: 'prince',
 3: 'little',
 4: 'pilot',
 5: 'rose',
 6: 'fox',
 7: 'young',
 8: 'planet',
 9: 'earth',
 10: 'story',
 11: 'plane',
 12: 'meets',
 13: 'asteroid',
 14: 'lessons',
 15: 'love',
 16: 'importance',
 17: 'written',
 18: 'antoine',
 19: 'saint-exupéry',
 20: 'poetic',
 21: 'tale',
 22: 'travels',
 23: 'home',
 24: 'begins',
 25: 'stranded',
 26: 'sahara',
 27: 'desert',
 28: 'crashes',
 29: 'trying',
 30: 'fix',
 31: 'mysterious',
 32: 'boy',
 33: 'comes',
 34: 'small',
 35: 'called',
 36: 'b-612',
 37: 'lives',
 38: 'alone',
 39: 'loves',
 40: 'deeply',
 41: 'recounts',
 42: 'journey',
 43: 'describing',
 44: 'visits',
 45: 'several',
 46: 'planets',
 47: 'inhabited',
 48: 'different',
 49: 'character',
 50: 'king',
 51: 'vain',
 52: 'man',
 53: 'drunkard',
 54: 'businessman',
 55: 'geographer',
 56: 'encounters',
 57: 'learns',
 58: 'valuable',
 59: 'responsibility',
 60: 'nature',
 61: 'adult',
 62: 'behavior',
 63: 'various',
 64: 'creatures',
 65: 'includin

In [ ]:
# 단어 빈도 확인
tokenizer.word_counts    # corpus의 전체 단어에 대한 빈도수를 OrderedDict 형태로 반환

OrderedDict([('little', 6),
             ('prince', 9),
             ('written', 1),
             ('antoine', 1),
             ('saint-exupéry', 1),
             ('poetic', 1),
             ('tale', 1),
             ('young', 2),
             ('travels', 1),
             ('home', 1),
             ('planet', 2),
             ('earth', 2),
             ('story', 2),
             ('begins', 1),
             ('pilot', 4),
             ('stranded', 1),
             ('sahara', 1),
             ('desert', 1),
             ('plane', 2),
             ('crashes', 1),
             ('trying', 1),
             ('fix', 1),
             ('meets', 2),
             ('mysterious', 1),
             ('boy', 1),
             ('comes', 1),
             ('small', 1),
             ('asteroid', 2),
             ('called', 1),
             ('b-612', 1),
             ('lives', 1),
             ('alone', 1),
             ('rose', 3),
             ('loves', 1),
             ('deeply', 1),
             ('recounts',

In [ ]:
# 문장(토큰 리스트)을 단어 인덱스로 치환해 정수 시퀀스 리스트로 변환
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)
sequences

[[3, 2, 1, 1, 1, 1, 1, 7, 2, 1, 1, 8, 9],
 [10, 1, 4, 1, 1, 1, 11, 1],
 [1, 1, 11, 12, 1, 7, 1, 3, 2],
 [3, 2, 1, 1, 13, 1, 1, 1, 1, 5, 1, 1],
 [1, 1, 4, 1, 1, 1, 1],
 [8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6],
 [1, 2, 1, 1, 14, 1, 1, 1, 1, 1],
 [9, 3, 2, 12, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1],
 [6, 1, 1, 1, 1, 1, 1, 1, 2, 1, 5],
 [1, 3, 2, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 4, 1, 1, 13, 1, 5],
 [10, 1, 4, 1, 14, 1, 3, 2, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

- 1<OOV>가 많은 이유는?
    - num_words=15 이기 때문에 상위 15개만 유효 단어로 사용하고, 나머지 단어들은 전부 OOV로 바뀌었다.
- num_words를 사용안하면?
    - tokenizer.word_index에 있는 모든 단어가 그대로 인덱싱된다.
        - 이 때 fit_on_texts 이후에 새로 등장하는 단어는 OOV로 가게 된다.
- num_words 쓰는 경우
    - 실제 모델 학습간 임베딩 테이블 크기 관리
    - 희귀 단어를 다 포함하면 노이즈 + 차원만 커짐 -> 일반화 성능이 떨어진다
    - 그래서 상위 N개의 단어만 쓰고 나머지는 OOV 처리하는것이 일반적임
- num_words의 사용범위
    - 아주 작은 실습 / 토이프로젝트 : num_words = 1,000 ~ 5,000
    - 간단한 분류 과제 : 5,000 ~ 20,000
    - 데이터가 아주 큰 경우 : 20,000 ~ 50,000